In [1]:
# Creating a classifier with CIFAR-10 dataset
# We will be using residual network architecture small version to see how it would perform 
# we will use below architecture for this problem as suggested in course

In [10]:
from IPython.display import IFrame
IFrame(src='https://www.cs.toronto.edu/~kriz/cifar.html', width=1000, height=600)


![resnetimage](https://user-images.githubusercontent.com/30661597/78585170-f4ac7c80-786b-11ea-8b00-8b751c65f5ca.PNG)

In [32]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
#create the dataset based on CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root = './', 
                             train=True, 
                             transform=transforms.ToTensor(),
                             download= False)
test_dataset = torchvision.datasets.CIFAR10(root = './', 
                             train=True, 
                             transform=transforms.ToTensor(),
                             download= True)

Files already downloaded and verified


In [3]:
#check the dataset attributes
print('Train dataset size : {}'.format(train_dataset.data.shape))
print('Test dataset size : {}'.format(train_dataset.data.shape))
print('# of classes : {}'.format(len(train_dataset.classes)))
print('classes are: {}'.format(train_dataset.classes))

Train dataset size : (50000, 32, 32, 3)
Test dataset size : (50000, 32, 32, 3)
# of classes : 10
classes are: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [4]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=10000,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=10000,
                                          shuffle=False)

In [30]:
#creating residual block
class Residual_block(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(Residual_block, self).__init__()
        self.conv1 = self.conv3x3(in_channels,out_channels,stride)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = self.conv3x3(out_channels, out_channels)
        self.downsample = downsample
        self.downsampling = self.conv3x3(in_channels,out_channels,stride=2)
        
    def conv3x3(self,in_channels,out_channels,stride=1):
        return nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,stride=stride,padding=1,bias=False)
    def forward(self, X):
        residual = X.clone()
        output = self.conv1(X)
        output = self.batchnorm(output)
        output = self.relu(output)
        output = self.conv2(output)
        output = self.batchnorm(output)
        if X.shape!= output.shape:
             residual = self.downsampling(X)
        output = output + residual
        output = self.relu(output)
        return output

In [21]:
#creating ResNet architecture with Residual blocks
# input to the contrctor should be Residual_block class and number of blocks in each layer
class ResNet(nn.Module):
    def __init__(self, Residual_block,in_channels= 3,no_classes=10):
        super(ResNet, self).__init__()
        self.layer1 = nn.Conv2d(in_channels=in_channels,out_channels=16,kernel_size=3,stride=1,padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.block1 = Residual_block(in_channels=16, out_channels=16)
        self.block2 = Residual_block(in_channels=16, out_channels=16)
        self.block3 = Residual_block(in_channels=16, out_channels=32,stride =2)
        self.block4 = Residual_block(in_channels=32, out_channels=32)
        self.block5 = Residual_block(in_channels=32, out_channels=64,stride =2)
        self.block6 = Residual_block(in_channels=64, out_channels=64)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64,no_classes)
    def forward(self,X):
        output = self.layer1(X)
        output = self.bn1(output)
        output = self.relu(output)
        output = self.block1(output)
        output = self.block2(output)
        output = self.block3(output)
        output = self.block4(output)
        output = self.block5(output)
        output = self.block6(output)
        output = self.avg_pool(output)
        output = output.view(output.size(0), -1)
        output = self.fc(output)
        return output

In [35]:
model = ResNet(Residual_block).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [33]:
X = torch.rand((1,3,32,32)).to(device)

In [ ]:
#Training the CNN
num_epochs = 5

#Define the lists to store the results of loss and accuracy
train_loss = []
train_accuracy = []


#Training
for epoch in range(num_epochs): 
    #Reset these below variables to 0 at the begining of every epoch
    correct = 0
    iterations = 0
    iter_loss = 0.0
    
    model.train()                   # Put the network into training mode
    
    for i, (inputs, labels) in enumerate(train_loader):
        
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()            # Clear off the gradient in (w = w - gradient)
        outputs = model(inputs)         
        loss = criterion(outputs, labels)  
        iter_loss += loss.item()       # Accumulate the loss
        loss.backward()                 # Backpropagation 
        optimizer.step()                # Update the weights
        
        # Record the correct predictions for training data 
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum()
        iterations += 1
    
    # Record the training loss
    train_loss.append(iter_loss/iterations)
    # Record the training accuracy
    train_accuracy.append((100 * correct / len(train_dataset)))
    print ('Epoch {}/{}, Training Loss: {:.3f}, Training Accuracy: {:.3f}'.format(epoch+1, num_epochs, train_loss[-1], train_accuracy[-1]))
    

In [ ]:
test_loss = []
test_accuracy = []
#Testing
    loss = 0.0
    correct = 0
    iterations = 0

    model.eval()                    # Put the network into evaluation mode
    
    for i, (inputs, labels) in enumerate(test_load):
        
        # Convert torch tensor to Variable
        inputs = Variable(inputs)
        labels = Variable(labels)
        
        CUDA = torch.cuda.is_available()
        if CUDA:
            inputs = inputs.cuda()
            labels = labels.cuda()
        
        outputs = model(inputs)     
        loss = loss_fn(outputs, labels) # Calculate the loss
        loss += loss.item()
        # Record the correct predictions for training data
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum()
        
        iterations += 1

    # Record the Testing loss
    test_loss.append(loss/iterations)
    # Record the Testing accuracy
    test_accuracy.append((100 * correct / len(test_dataset)))
    
    